In [1]:
import json
import os
import firebase_admin
from firebase_admin import credentials, firestore

In [2]:
cred = credentials.Certificate(os.getcwd() + "/cogs121-goduck-firebase-adminsdk-f5baa-73bc1269cd.json")
firebase_admin.initialize_app(cred)

In [3]:
db = firestore.client()